# This ntb looks at foreground images of control group of mice getting to know the typical background fluorescence of mice

Content
* [FRG](#FRG)
  * [Get correction matrices for background and mice](#Get-correction-matrices-for-background-and-mice)
    * [empty slots](#Get-background-on-empty-positions)
    * [average mouse signal accounting for empty slot singal](#Get-the-background-of-mice-accounting-for-the-empty-slot-background)
    * [Do the same on the entire dataset](#Do-the-same-on-the-entire-dataset)

## FRG

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import h5py
import os, sys

# Importing module for simulations checks
functions_modul =  "./functions"
sys.path.insert(0, functions_modul)

from load_h5_into_np import load_h5_into_np


file_path = "/Users/jan/Documents/PhD/Ondra/Ondra_data/control/frg"
files = os.listdir(file_path)

In [ ]:
fig, ax = plt.subplots(2 * len(files), 5, figsize=(40, 40))
off = 70
background = 5
bcg_mean_705 = np.zeros((len(files), 5))

for j in range(len(files)):
    # load the arr
    tmp_arr = load_h5_into_np(os.path.join(file_path, files[j]))

    for i in range(5):
        y = tmp_arr[off + (200 * i) : off + (200 * (i + 1)), 250:-1]
        ax[2 * j, i].imshow(y > background)
        #         print('(',off+(200*i),',',off+(200*(i+1)),',',250,',',-1,')')
        print("mean ", y.mean())
        bcg_mean_705[j, i] = y.mean()
        sns.histplot((y[y > background]).reshape(-1), ax=ax[2 * j + 1, i])


plt.savefig(f"frg_control_{off}_th{background}.png")
# plt.savefig(f"frg_control_{off}_th{background}.pdf")

### Get correction matrices for background and mice

Automatic way of detecting mice is not working well because the mouse has such a weak signal that the fourth slot has always the highest intensity regardless whether there is a mouse or not...

Therefore it has to be checked manually, the name is extracted and the position fit based on the name for the individual mouse. 

### Get background on empty positions

In [ ]:
# fig, ax = plt.subplots(2*len(files),5, figsize=(40,40))
off = 70
background = 5
bcg_mean_705 = np.zeros((len(files), 5))

positions = [[], [], [], [], []]


for j, name in enumerate(files):
    # load the arr
    tmp_arr = load_h5_into_np(os.path.join(file_path, files[j]))

    for i in range(5):
        y = tmp_arr[off + (200 * i) : off + (200 * (i + 1)), 250:-1]
        #         plt.imshow(y>background)
        #         print('(',off+(200*i),',',off+(200*(i+1)),',',250,',',-1,')')
        print("mean ", y.mean())
        bcg_mean_705[j, i] = y.mean()
    #         sns.histplot((y[y>background]).reshape(-1), ax=ax[2*j+1,i])

    if "vsechny" in name:
        # get indices of three slots with mice
        # https://stackoverflow.com/questions/6910641/how-do-i-get-indices-of-n-maximum-values-in-a-numpy-array
        inds = np.argpartition(bcg_mean_705[j, :], -3)[-3:]

        # plot full positions
        for k in inds:
            z = tmp_arr[off + (200 * k) : off + (200 * (k + 1)), 250:-1]
            plt.imshow(z > background)
            plt.show()
            print(f"Mouse {j} slot {k}")

        # get the empty positions and append their values to corresponding list
        for empty in set(range(5)) - set(inds):
            positions[empty].append(
                tmp_arr[off + (200 * empty) : off + (200 * (empty + 1)), 250:-1]
            )

    elif "mys" in name:
        # get the position manually, CAUTION!!! Must be checked by a user
        k = int(name.split("_")[5][-1])
        print(f"Mouse {j} slot {k}")
        z = tmp_arr[off + (200 * k) : off + (200 * (k + 1)), 250:-1]
        plt.imshow(z > background)
        plt.show()
        # get the empty positions and append their values to corresponding list
        for empty in set(range(5)) - set(
            [k]
        ):  # k is always an int, so it must be a list
            positions[empty].append(
                tmp_arr[off + (200 * empty) : off + (200 * (empty + 1)), 250:-1]
            )

# create an array of means for each position
correction_matrix_slots_list = np.concatenate(
    [np.asarray(positions[i]).mean(axis=0) for i in range(len(positions))]
)

In [ ]:
positions[0]

In [ ]:
correction_matrix_slots = np.concatenate(
    [np.asarray(positions[i]).mean(axis=0) for i in range(len(positions))]
)
print(correction_matrix_slots.shape)

# plot the data with different thresholds
fig, axs = plt.subplots(ncols=4, figsize=(24, 5))

thr_list = [5, 10, 20, 50]

for thr, ax in zip(thr_list, axs.flatten()):
    ax.imshow(correction_matrix_slots < thr)
    ax.set_title(f"thr = {thr}")

### Get the background of mice accounting for the empty slot background

In [ ]:
# fig, ax = plt.subplots(2*len(files),5, figsize=(40,40))
off = 70
background = 5
bcg_mean_705 = np.zeros((len(files), 5))

# get average signal on mice
m_positions = [[], [], [], [], []]

for j, name in enumerate(files):
    # load the arr
    tmp_arr = load_h5_into_np(os.path.join(file_path, files[j]))

    for i in range(5):
        y = tmp_arr[off + (200 * i) : off + (200 * (i + 1)), 250:-1]
        # correct for the given slot
        y -= correction_matrix_slots[(200 * i) : (200 * (i + 1)), :]
        # remove negative values
        y[y < 0] = 0

        #         plt.imshow(y>background)
        print("mean ", y.mean())
        bcg_mean_705[j, i] = y.mean()
    #         sns.histplot((y[y>background]).reshape(-1), ax=ax[2*j+1,i])

    if "vsechny" in name:
        # get indices of three slots with mice
        # https://stackoverflow.com/questions/6910641/how-do-i-get-indices-of-n-maximum-values-in-a-numpy-array
        inds = np.argpartition(bcg_mean_705[j, :], -3)[-3:]

        # plot full positions
        for k in inds:
            z = tmp_arr[off + (200 * k) : off + (200 * (k + 1)), 250:-1]
            #             plt.imshow(z>background)
            print(f"Mouse {j} slot {k}")
            #             plt.show()

            # get the full positions and append their values to corresponding list
            m_positions[k].append(z)

    elif "mys" in name:
        # get the position manually, CAUTION!!! Must be checked by a user
        k = int(name.split("_")[5][-1])
        print(f"Mouse {j} slot {k}")
        z = tmp_arr[off + (200 * k) : off + (200 * (k + 1)), 250:-1]
        #         plt.imshow(z>background)
        #         plt.show()
        # get the empty positions and append their values to corresponding list
        m_positions[k].append(z)


# create an array of means for each position if mouse exist, else use background signal
correction_matrix_slots_mice_list = np.concatenate(
    [
        np.asarray(m_positions[i]).mean(axis=0)
        if m_positions[i]
        else np.asarray(positions[i]).mean(axis=0)
        for i in range(len(m_positions))
    ]
)

### Use max instead of mean to get the background signal -- eventually not used and not implemented

How this works is that it subtracts the background noise but only to get the mean of that and subsequently take the slots with highest remaining mean (assuming that we removed the bacground). BUT the signal is taken from the original matrix, ie WITHOUT the background subtraction which means that it has BOTH the mouse and the background accounted for.

The problem is that the location of the specs are not fixed and move a bit. If that happens, the algorithm fails on that (see New way below). A solution would be to take max value in the background instead of a mean, that should remove the entire background.


The full cleaning, and I'm getting more and more positive about that, should be that:
1) take the background in all empty slots, take its max. Does not have to be on controls only, could actually be done on the entire dataset (that should work because the mouse selection there is much easier -- their intensities are much higher so it's gonna be easy enough to pick the spots automatically and not have false positive with the spill; a workaround is to take only the 3 mice pictures). This will serve as the correction matrix for background. Why to take max and not mean or whatever else? In short, the positins of the mice/slots/device are not constant and shift a bit in space. That means that if we then take mean, the actuall values are diminished and the leftovers after subtracton can still cause the artefact we want to prevent. See the example above. However, max will cover this possibility because if we have enough of images for the background, it will hopefully cover all possible position and then the max will appear in the correction matrix. The price for that is being too harsh and potentially losing signal by subtracting it -- but it's a region we don't care about.

2) Having that, one should take the *control* group only. For each image and slot subtract the corresponding max background (of the given slot), take the slot with a mouse and average those out. I'm unsure here if one should still do it slot specific (still accounts for some local problems) or average across slots (better statistics) but probably the first. This will give a background specific to a mouse on a given position (or generally a mouse) without the background! Again, WITHOUT A BACKGROUND, must be subtracted, not like before. That will yield a mouse specific correction matrix.

3) When doing the analysis, one should load and image, sutract background AND a mouse on a given position and proceed with the analysis.

In [ ]:
# create an array of means for each position
correction_matrix_slots_list_max = np.concatenate(
    [np.asarray(positions[i]).max(axis=0) for i in range(len(positions))]
)
plt.imshow(correction_matrix_slots_list_max>30)

### Do the same on the entire dataset

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import h5py
import os

# all_files_path = "/Users/jan/Documents/PhD/Ondra/Ondra_data/frg-h5"
# all_files_path = "/Users/jan/Documents/PhD/Ondra/Ondra_data/frg-h5-old"
all_files_path = "/Users/jan/Documents/PhD/Ondra/Ondra_data/frg"
all_files = os.listdir(all_files_path)
control_files = [x for x in all_files if "kontrola" in x]
control_files

In [ ]:
# check shape
for j, name in enumerate(control_files):
    # load the arr
    tmp_arr = load_h5_into_np(os.path.join(all_files_path, control_files[j]))
    if tmp_arr.shape != (1024, 1024):
        print("\n\n", name, "\n\n")

In [ ]:
# fig, ax = plt.subplots(2*len(files),5, figsize=(40,40))
off = 70
background = 5
positions_all = [[], [], [], [], []]
bcg_mean_705 = np.zeros((len(control_files), len(positions_all)))

for j, name in enumerate(control_files):
    # load the arr
    tmp_arr = load_h5_into_np(os.path.join(all_files_path, control_files[j]))
    # loop only over correct shape
    if tmp_arr.shape == (1024, 1024):
        for i in range(5):
            y = tmp_arr[off + (200 * i) : off + (200 * (i + 1)), 250:-1]
            #             plt.imshow(y>background)
            #         print('(',off+(200*i),',',off+(200*(i+1)),',',250,',',-1,')')
            print("mean ", y.mean())
            bcg_mean_705[j, i] = y.mean()
        #         sns.histplot((y[y>background]).reshape(-1), ax=ax[2*j+1,i])

        if "vsechny" in name:
            # get indices of three slots with mice
            # https://stackoverflow.com/questions/6910641/how-do-i-get-indices-of-n-maximum-values-in-a-numpy-array
            inds = np.argpartition(bcg_mean_705[j, :], -3)[-3:]

            # plot full positions
            for k in inds:
                print(f"Mouse {j} slot {k}")
                z = tmp_arr[off + (200 * k) : off + (200 * (k + 1)), 250:-1]
                plt.imshow(z > background)
                plt.show()

            # get the empty positions and append their values to corresponding list
            for empty in set(range(5)) - set(inds):
                positions_all[empty].append(
                    tmp_arr[off + (200 * empty) : off + (200 * (empty + 1)), 250:-1]
                )

        elif "mys" in name:
            # get the position manually, CAUTION!!! Must be checked by a user
            k = int(name.split("_")[5][-1])
            print(f"Mouse {j} slot {k}")
            z = tmp_arr[off + (200 * k) : off + (200 * (k + 1)), 250:-1]
            plt.imshow(z > background)
            plt.show()
            # get the empty positions and append their values to corresponding list
            for empty in set(range(5)) - set(
                [k]
            ):  # k is always an int, so it must be a list
                positions_all[empty].append(
                    tmp_arr[off + (200 * empty) : off + (200 * (empty + 1)), 250:-1]
                )

In [ ]:
# create an array of means for each position
correction_matrix_slots_all = np.concatenate(
    [np.asarray(positions_all[i]).mean(axis=0) for i in range(len(positions_all))]
)

print(correction_matrix_slots_all.shape)

# plot the data with different thresholds
fig, axs = plt.subplots(ncols=4, figsize=(24, 5))

thr_list = [5, 10, 20, 50]

for thr, ax in zip(thr_list, axs.flatten()):
    ax.imshow(correction_matrix_slots_all < thr)
    ax.set_title(f"thr = {thr}")
plt.suptitle(
    "Correction matrix of background fluorescence under different threshold",
    fontsize=20,
)
plt.show()

In [ ]:
off = 70
background = 5
m_positions_all = [[], [], [], [], []]
bcg_mean_705 = np.zeros((len(control_files), len(positions_all)))

for j, name in enumerate(control_files):
    # load the arr
    tmp_arr = load_h5_into_np(os.path.join(all_files_path, control_files[j]))
    # loop only over correct shape
    if tmp_arr.shape == (1024, 1024):
        for i in range(5):
            y = tmp_arr[off + (200 * i) : off + (200 * (i + 1)), 250:-1]
            # correct for the given slot
            y -= correction_matrix_slots[(200 * i) : (200 * (i + 1)), :]
            # remove negative values
            y[y < 0] = 0
            #         plt.imshow(y>background)
            print("mean ", y.mean())
            bcg_mean_705[j, i] = y.mean()
        #         sns.histplot((y[y>background]).reshape(-1), ax=ax[2*j+1,i])

        if "vsechny" in name:
            # get indices of three slots with mice
            # https://stackoverflow.com/questions/6910641/how-do-i-get-indices-of-n-maximum-values-in-a-numpy-array
            inds = np.argpartition(bcg_mean_705[j, :], -3)[-3:]

            # plot full positions
            for k in inds:
                print(f"Mouse {j} slot {k}")
                z = tmp_arr[off + (200 * k) : off + (200 * (k + 1)), 250:-1]
                plt.imshow(z > background)
                plt.show()

                # get the full positions and append their values to corresponding list
                m_positions_all[k].append(z)

        elif "mys" in name:
            # get the position manually, CAUTION!!! Must be checked by a user
            k = int(name.split("_")[5][-1])
            print(f"Mouse {j} slot {k}")
            z = tmp_arr[off + (200 * k) : off + (200 * (k + 1)), 250:-1]
            plt.imshow(z > background)
            plt.show()
            # get the empty positions and append their values to corresponding list
            m_positions_all[k].append(z)

In [ ]:
# create an array of means for each position if mouse exist, else use background signal
correction_matrix_slots_mice_all = np.concatenate(
    [
        np.asarray(m_positions_all[i]).mean(axis=0)
        if m_positions_all[i]
        else np.asarray(positions_all[i]).mean(axis=0)
        for i in range(len(m_positions_all))
    ]
)

print(correction_matrix_slots_mice_all.shape)

# plot the data with different thresholds
fig, axs = plt.subplots(ncols=4, figsize=(24, 5))

thr_list = [5, 10, 20, 50]

for thr, ax in zip(thr_list, axs.flatten()):
    ax.imshow(correction_matrix_slots_mice_all < thr)
    ax.set_title(f"thr = {thr}")

plt.suptitle(
    "Correction matrix with correction for mice's fluorescence under different threshold",
    fontsize=20,
)
plt.show()

In [ ]:
sns.heatmap(correction_matrix_slots_mice_all)

In [ ]:
plot_thr = 100

plt.hist(
    correction_matrix_slots_all[
        (0 < correction_matrix_slots_all) & (correction_matrix_slots_all < plot_thr)
    ].reshape(-1),
    bins=1000,
)
plt.axvline(
    correction_matrix_slots_all.mean(),
    c="k",
    label="mean {:.3}".format(correction_matrix_slots_all.mean()),
)
plt.title(
    f"Histogram of intensity values in correction matrix without mice for values <{plot_thr}"
)
plt.xlabel("Intensity")
plt.ylabel("Count")
plt.legend()
plt.show()

In [ ]:
plt.hist(
    correction_matrix_slots_mice_all[correction_matrix_slots_mice_all > plot_thr].reshape(-1),
    bins=1000,
)
plt.title(
    f"Histogram of intensity values in correction matrix with mice for values >{plot_thr}"
)
plt.xlabel("Intensity")
plt.ylabel("Count")
plt.show()

In [ ]:
# save

np.save("./correction_matrix_slots_mice_all.npy", correction_matrix_slots_mice_all)

In [ ]:
%load_ext watermark

%watermark -a 'Jan Kadlec' -nmvu -iv